In [3]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Define dataset paths
base_dir = '/home/mac/Pictures/Landmarks'

# Image parameters
img_height, img_width = 224, 224
batch_size = 32

# Image data generator with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    validation_split=0.2,  # 20% of data for validation
    horizontal_flip=True,
    zoom_range=0.2,
    shear_range=0.2
)

train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # Training data
)

val_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # Validation data
)

# Load pre-trained ResNet50 model with weights trained on ImageNet
base_model = tf.keras.applications.ResNet50(
    weights='imagenet', 
    input_shape=(img_height, img_width, 3),
    include_top=False  # Exclude the fully connected layer at the top
)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Build the model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),  # Converts feature maps to a single vector per image
    layers.Dense(1024, activation='relu'),  # Dense layer
    layers.Dropout(0.5),  # Dropout for regularization
    layers.Dense(train_generator.num_classes, activation='softmax')  # Output layer for classification
])

# Compile the model
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

# Model summary
model.summary()

# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10  # Set more epochs for better results
)

# Evaluate the model
val_loss, val_acc = model.evaluate(val_generator)
print(f'Validation Accuracy: {val_acc * 100:.2f}%')

# Class labels
class_labels = list(train_generator.class_indices.keys())

# Predict on validation data and print classification report
val_generator.reset()
preds = model.predict(val_generator)
y_pred = np.argmax(preds, axis=1)
y_true = val_generator.classes

# Classification report
print(classification_report(y_true, y_pred, target_names=class_labels))

# Save the model
model.save('landmark_recognition_model.h5')


Found 54 images belonging to 3 classes.
Found 12 images belonging to 3 classes.


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1024)           │     2,098,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 3)              │         3,075 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 25,688,963 (98.00 MB)

 Trainable params: 2,101,251 (8.02 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

Epoch 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 11s 4s/step - accuracy: 0.3036 - loss: 1.8578 - val_accuracy: 0.3333 - val_loss: 1.9905
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 3s/step - accuracy: 0.3159 - loss: 1.7694 - val_accuracy: 0.3333 - val_loss: 1.1204
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 3s/step - accuracy: 0.3056 - loss: 1.3958 - val_accuracy: 0.3333 - val_loss: 1.5771
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - accuracy: 0.3264 - loss: 1.7605 - val_accuracy: 0.3333 - val_loss: 1.0894
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 3s/step - accuracy: 0.3653 - loss: 1.2805 - val_accuracy: 0.3333 - val_loss: 1.2296
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - accuracy: 0.2562 - loss: 1.5122 - val_accuracy: 0.4167 - val_loss: 1.1320
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - accuracy: 0.3349 - loss: 1.2601 - val_accuracy: 0.4167 - val_loss: 1.0586
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - accuracy: 0.3453 - loss: 1.2774 - val_accuracy: 0.4167 - val_loss: 1.1906
Epoch 9/10
2/2 

/home/mac/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mac/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mac/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                   precision    recall  f1-score   support

     Eiffel Tower       0.00      0.00      0.00         4
Statue of Liberty       0.00      0.00      0.00         4
        Taj Mahal       0.20      0.50      0.29         4

         accuracy                           0.17        12
        macro avg       0.07      0.17      0.10        12
     weighted avg       0.07      0.17      0.10        12



In [1]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the trained model from the saved file
model = load_model('landmark_recognition_model.h5')

# Function to load and preprocess an image for prediction
def predict_landmark(image_path):
    img = image.load_img(image_path, target_size=(224, 224))  # Load the image
    img_array = image.img_to_array(img) / 255.0               # Convert image to array and normalize
    img_array = np.expand_dims(img_array, axis=0)             # Add batch dimension

    # Predict the landmark
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions)  # Get the index of the class with the highest score

    # Define the class labels (these should match your folder names during training)
    class_labels = ['Eiffel_Tower', 'Great_Wall_of_China', 'Statue_of_Liberty']  # Add more as needed
    return class_labels[predicted_class]

# Test with a new image
image_path = 'path_to_new_statue_of_liberty_image.jpg'
predicted_landmark = predict_landmark(image_path)
print(f'The predicted landmark is: {predicted_landmark}')


2024-10-22 18:55:45.340940: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-22 18:55:45.349753: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-22 18:55:45.412269: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-22 18:55:45.470835: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-22 18:55:45.521242: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

FileNotFoundError: [Errno 2] No such file or directory: 'path_to_new_statue_of_liberty_image.jpg'